In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os

from graph_utils import graph_gen, find_alpha_lower_bound
from problem import Synthetic

from algorithms import IPLUX, UDC
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')



# network

num_node = 5
num_edge = 7

np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

# problem data

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

Graph with 5 nodes and 7 edges
(5, 5)


In [12]:
# load instance 
instance_name = 'instance1_N20E40'
instance_dir = 'instance/' + instance_name

# network
network = np.load(instance_dir + '/graph/subgraph_W.npy')
num_node = network.shape[0]
print(network.shape)

# problem data
parameters = {}
parameters['N'] = num_node
parameters['d'] = 1 # temporary, reset in load()
parameters['m'] = 1 # temporary
parameters['p'] = 1 # temporary

prob = Synthetic(parameters)
prob.save_dir = instance_dir + '/problem'
prob.load()

alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(20, 20)
loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -1.7937811849951981
x* [[ 1.69e-08 -4.54e-02 -1.25e-02]
 [-1.97e-11  2.95e-10  1.96e-01]
 [-1.55e-01 -1.82e-10 -4.31e-02]
 [ 2.21e-10  6.41e-11 -2.57e-01]
 [ 5.46e-01 -3.09e-01 -2.26e-10]
 [-7.96e-10  4.77e-01 -9.77e-02]
 [ 2.30e-01  9.67e-10 -4.08e-02]
 [ 4.42e-10  5.04e-01  2.56e-02]
 [-4.79e-01 -1.56e-09  3.47e-10]
 [ 1.17e-10 -9.51e-11 -4.60e-01]
 [ 4.42e-10 -4.19e-01  1.77e-01]
 [-1.46e-09  6.04e-10 -1.04e-09]
 [-3.70e-01  6.45e-01  3.92e-10]
 [ 1.95e-10 -2.86e-10 -4.45e-10]
 [-1.46e-08 -3.48e-01  8.41e-10]
 [-8.32e-10 -1.66e-01  1.02e-09]
 [ 1.29e-09 -2.08e-01 -1.63e-01]
 [ 2.99e-10  4.11e-11  7.93e-10]
 [ 1.05e-01  3.94e-10 -3.33e-01]
 [ 8.19e-10 -5.60e-10 -4.09e-10]]
||x*|| 1.617194870033797



In [3]:
MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
rho_choice = [0.1, 0.5, 2, 5]
alpha_choice = [alpha_lower+6]
rho_choice = [2]


np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
        for i in range(MAX_ITER):
            alg.step()

self.prob minimize param837 @ var836 + norm1(var836) + 4.5 @ QuadForm(var836 + -param838, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) + 0.25 @ quad_over_lin(param839 @ var836, 1.0) + param840 @ var836 + param841[0] @ QuadForm(var836, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) + Promote(-2.0, (3,)) @ param842 @ var836
subject to QuadForm(var836 + -param844, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) <= param845[0]
reset


ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (5,)

In [22]:
MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


alpha_choice = [0, 2, 4, 8, 10]
rho_choice = [0.1, 0.5, 2, 5]
alpha_choice = [0]
rho_choice = [1, 3]

# alg = UDC(prob, network, rho=1, alpha=1, param_setting='proximal_tracking', verbose=False)


# for i in range(MAX_ITER):
#     alg.step()
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        alg = UDC(prob, network, rho=rho, alpha=alpha, param_setting='proximal_tracking', verbose=False)
        for i in range(MAX_ITER):
            alg.step()


UDC setting: proximal_tracking
self.prob minimize quad_over_lin(param6807 @ var6806, 1.0) + param6808 @ var6806 + param6809[0] @ norm1(var6806) + 0.0 @ quad_over_lin(Promote(param6810[0], (3,)) @ var6806 + -param6811, 1.0) + 0.5 @ quad_over_lin(maximum(param6812 + quad_over_lin(var6806 + -param6814, 1.0) + -param6815, 0.0), 1.0) + 0.5 @ quad_over_lin(param6813 + param6818 @ var6806, 1.0)
subject to QuadForm(var6806 + -param6816, [[ 1.00e+00 0.00e+00 0.00e+00]
 [0.00e+00  1.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  1.00e+00]]) <= param6817[0]
reset
UDC_pt alpha 0 rho 1, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

UDC_pt alpha 0 rho 1, iter 100, obj err: 1.72e-02, cons vio: 3.15e-02
time 44.04, saved

UDC_pt alpha 0 rho 1, iter 200, obj err: 1.19e-03, cons vio: 2.89e-03
time 91.13, saved

UDC_pt alpha 0 rho 1, iter 300, obj err: 2.50e-04, cons vio: 4.76e-04
time 141.67, saved

UDC_pt alpha 0 rho 1, iter 400, obj err: 9.83e-05, cons vio: 1.80e-04
time 195.63, saved

UDC_pt

In [26]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |

from plot_utils import MyFigure

# alg_name = 'IPLUX'
alg_name = 'UDC_pt'

# alpha_choice = [alpha_lower, alpha_lower+2, alpha_lower+4, alpha_lower+6]
# rho_choice = [0.1, 0.5, 2, 5, 10]
# rho_choice = [0.05, 0.1, 0.5, 2, 5]
# MAX_ITER = 2000
alpha_choice = [0, 2, 4, 8, 10]
# rho_choice = [0.5, 1, 2, 3]
rho_choice = [2]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
# marker = [".", "o", "^", "s", "p", "P", "*"]
linestyle = ['-', '--', '-.', ':', '']



# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$|\sum_{i=1}^N f_i(x_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)|$',
                            # xlabel = 'iteration'
                            ylabel='objective error',
                            yscale='log')

cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')

x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='x distance',
                            yscale='log')

# obj_err_figure.add_line("IPLUX", alg.obj_err_log)
for rho in rho_choice:
    for alpha in alpha_choice:
        ci = alpha_choice.index(alpha)
        lsi = rho_choice.index(rho)
        # ci = rho_choice.index(rho)
        # lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        prefix = f'{alg_name}_a{alpha}_r{rho}'
        filename = f'log/N{num_node}/{prefix}_oe.txt'
        
        obj_err_figure.add_line_file(prefix, filename, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename, style=c+ls)
        
obj_err_figure.paint(MAX_ITER=MAX_ITER)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


TypeError: draw_wrapper() missing 1 required positional argument: 'renderer'